In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [25]:
reviews_url = "https://www.amazon.in/boAt-Cancelling-Bluetooth-Headphones-Immersive/product-reviews/B09MTRDQB5/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

In [26]:
len_page = 10

In [27]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):

    # Empty List define to store all pages html data
    soups = []

    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):

        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }

        # Request make for each page
        response = requests.get(url, headers=headers)

        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')

        # Add single Html page data in master soups list
        soups.append(soup)

    return soups

In [28]:
#Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []

    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')

    # Iterate all Reviews BOX
    for box in boxes:

        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)

    return data_dicts

In [29]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

# Empty List to Hold all reviews data
reviews = []

# Iterate all Html page
for html_data in html_datas:

    # Grab review data
    review = getReviews(html_data)

    # add review data in reviews empty list
    reviews += review

# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [30]:
df_reviews

,Name,Stars,Title,Date,Description
0,Kindle Customer,5.0,5.0 out of 5 stars\nImpressive quality,N/A,I bought this for my daughter - and 2 days lat...
1,Prathmesh P.,4.0,4.0 out of 5 stars\nPros and Cons,N/A,Pros:1. Beautiful sound quality2. Crystal clea...
2,Udipta sahu,5.0,5.0 out of 5 stars\nBoat 751ANC vs JBL 760NC,N/A,The media could not be loaded.\n ...
3,Nidhi,1.0,1.0 out of 5 stars\ntoooo heavy,N/A,the headphones are too heavy cannot wear for m...
4,Saurabh Singh,5.0,5.0 out of 5 stars\nbest earphones i have ever...,N/A,build quality not too great infact i had to st...
...,...,...,...,...,...
75,Rakshit kulhari,3.0,3.0 out of 5 stars\nProduct seems fair,N/A,"Mic quality is not so good , have to speak lou..."
76,Mohit Jain,4.0,"4.0 out of 5 stars\nBlessing in disguise, orde...",N/A,Don't order the blue color of these headphones...
77,Shanmukh,5.0,5.0 out of 5 stars\nExcellent sound! Immersive...,N/A,Everything is fine with the headphones. Signat...
78,Xyz,4.0,4.0 out of 5 stars\nAudio quality is meh!!! Bo...,N/A,Good looksSounds above averageBuild is strongA...
